# Clean all_communities_data

**Résumé des étapes:**
* Suppression de valeurs nulles
    * Colonnes et lignes
    * Suppression doublons
* Siren
    * Clean et formatage
    * Suppression doublons (type, nom)
    * Ajout champ identifiant(siren+type)
* Code region
    * Formatage
    * complétion 
* Code département
    * formatage
    * Complétion
* Code cog
    * Formatage
    * complétion
* enrichir avec latitude et longitude
    * Ajout pour région et département (seule source trouvé)

In [734]:
import numpy as np
import pandas as pd 

In [735]:
path = '../data/datasets/'

In [736]:
communities_df = pd.read_csv(path + 'all_communities_data.csv', delimiter=';', low_memory=False, index_col=False)

In [737]:
communities_df.head()

,Unnamed: 0,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,url_ptf,url_datagouv,id_datagouv,merge,ptf,trancheeffectifsunitelegale,effectifssup50
0,0,Guadeloupe,239710015,REG,1,NaN,NaN,NaN,NaN,396153.0,NaN,https://regionguadeloupe.opendatasoft.com/expl...,NaN,NaN,ptf,KaruData,42.0,True
1,1,Île-de-France,237500079,REG,11,NaN,NaN,NaN,NaN,12291279.0,NaN,https://data.iledefrance.fr/explore/?refine.pu...,https://www.data.gouv.fr/fr/organizations/534f...,534fffa8a3a7292c64a780c8,ptf+datagouv,"Ile-de-France, DataGouv",53.0,True
2,2,Martinique,200055507,CTU,2,NaN,NaN,NaN,NaN,377711.0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,True
3,3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,NaN,NaN,2634852.0,NaN,https://data.centrevaldeloire.fr/explore/?sort...,https://www.data.gouv.fr/fr/organizations/regi...,561a7f00c751df4e6dcdbb48,ptf+datagouv,"Centre Val-de-Loire, DataGouv",51.0,True
4,4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,NaN,NaN,2885864.0,NaN,https://trouver.ternum-bfc.fr/organization/reg...,https://www.data.gouv.fr/fr/organizations/5835...,5835cdb988ee383e0bc65bb3,ptf+datagouv,"Bourgogne-Franche-Comté, DataGouv",51.0,True


In [738]:
communities_df.shape

(36353, 18)

In [739]:
(communities_df.isna().sum()/communities_df.shape[0])*100

Unnamed: 0                      0.000000
nom                             0.008252
siren                           0.000000
type                            0.002751
cog                             3.488020
cog_3digits                    99.730421
code_departement                0.324595
code_departement_3digits        0.324595
code_region                     0.055016
population                      0.002751
epci                            3.930900
url_ptf                        98.910681
url_datagouv                   98.324760
id_datagouv                    98.407284
merge                          97.678321
ptf                            97.758094
trancheeffectifsunitelegale     5.801447
effectifssup50                  0.000000
dtype: float64

## Suppression des colonnes à valeurs nulles (>=90%)

In [740]:
communities_df.drop(['cog_3digits','url_ptf','url_datagouv','id_datagouv','merge','ptf',"Unnamed: 0"], axis=1, inplace=True)

In [741]:
communities_df.shape

(36353, 11)

## Suppression des lignes où les champs 'nom', 'type' et siren sont absents

In [742]:
communities_df.dropna(subset=['nom','type','siren'], inplace=True)

In [743]:
(communities_df.isna().sum()/communities_df.shape[0])*100

nom                            0.000000
siren                          0.000000
type                           0.000000
cog                            3.485557
code_departement               0.316369
code_departement_3digits       0.316369
code_region                    0.046768
population                     0.000000
epci                           3.928473
trancheeffectifsunitelegale    5.799175
effectifssup50                 0.000000
dtype: float64

## doublons

In [744]:
communities_df[communities_df.duplicated(keep=False)]

,nom,siren,type,cog,code_departement,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50
25000,Aincille,216400119,COM,64011,64,064,75.0,118.0,200067106.0,2.0,False
25001,Aincille,216400119,COM,64011,64,064,75.0,118.0,200067106.0,2.0,False
36002,CA du Pays de l'Or,243400470,CA,NaN,34,034,76,45153.0,NaN,NaN,False
36003,CA du Pays de l'Or,243400470,CA,NaN,34,034,76,45153.0,NaN,NaN,False


In [745]:
communities_df.drop_duplicates(inplace=True)

In [746]:
communities_df.shape

(36348, 11)

## Siren

### Clean siren et formatage

In [747]:
from cleanup.cleanup_siret_functions import clean_numeros

In [748]:
communities_df["siren"] = communities_df["siren"].apply(clean_numeros)

## doublons siren

In [749]:
communities_df[communities_df["siren"].duplicated(keep=False)]

,nom,siren,type,cog,code_departement,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50
85,Métropole de Lyon,200046977,DEP,691,NaN,NaN,84,1402326.0,NaN,52.0,True
91,Paris,217500016,DEP,75,NaN,NaN,11,2204773.0,NaN,53.0,True
29389,Paris,217500016,COM,75056,75,075,11.0,2204773.0,200054781.0,53.0,True
35348,Métropole de Lyon,200046977,MET,NaN,69,069,84,1402326.0,NaN,52.0,True
35818,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,24566.0,NaN,NaN,False
35819,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,1157126.0,NaN,NaN,False
35820,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,1181692.0,NaN,NaN,False
35822,CA Agglomération d'Agen,200096956,CA,NaN,47,047,75,5372.0,NaN,NaN,False
35823,CA Agglomération d'Agen,200096956,CA,NaN,47,047,75,99115.0,NaN,NaN,False


### Remarque : 
Les doublons de SIREN sont dus aux types différents (ce qui ne pose pas de problème) ou à la population.\
Concernant la population, je supprime les doublons, mais la population n'est pas forcement correct.\
siren de Métropole Européenne de Lille : 200093201\
siren de CA Agglomération d'Agen : 200096956\

In [750]:
communities_df.drop_duplicates(subset=["nom","siren","type"], inplace=True)

In [751]:
communities_df.shape

(36345, 11)

### Créer un identifiant à base de siren et de type

In [752]:
communities_df["id"] = communities_df["type"].apply(lambda x: x.lower())+communities_df['siren']

In [753]:
communities_df = communities_df.set_index('id', drop=False)


## Code region

### Formatage

In [754]:
communities_df['code_region'] = communities_df['code_region'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)


### compléter les codes régions
source NocoDB : ofgl-base-regions-consolidee.csv

In [755]:
communities_df[communities_df['code_region'].isna()]["type"].value_counts()

REG    14
CTU     3
Name: type, dtype: int64

In [756]:
base_regions = pd.read_csv(path + 'ofgl-base-regions-consolidee.csv', delimiter=';')

In [757]:
base_regions.head()

,exer,outre_mer,reg_is_ctu,reg_code,reg_name,categ,siren,lbudg,agregat,montant_bp,...,ordre_analyse1_section2,ordre_analyse1_section3,ordre_analyse2_section1,ordre_analyse2_section2,ordre_analyse2_section3,ordre_analyse3_section1,ordre_analyse3_section2,ordre_analyse3_section3,ordre_analyse4_section1,annee_join
0,2014,Non,Non,76,Occitanie,REG,233100015,REG MIDI-PYRENEES,Achats et charges externes,89130711.96,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2014
1,2014,Non,Non,84,Auvergne-Rhône-Alpes,REG,236300018,REG AUVERGNE,Achats et charges externes,38450351.83,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2014
2,2014,Non,Non,75,Nouvelle-Aquitaine,REG,233300011,REG AQUITAINE,Autres dotations de fonctionnement,67333779.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014
3,2014,Non,Non,53,Bretagne,REG,233500016,REG BRETAGNE,Autres dotations de fonctionnement,71885736.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014
4,2014,Non,Non,44,Grand Est,REG,236700019,REG ALSACE,Autres dotations de fonctionnement,66042585.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014


In [758]:
base_regions['siren'] = base_regions['siren'].apply(clean_numeros)
base_regions['reg_code'] = base_regions['reg_code'].astype(str)
communities_df['code_region'] = communities_df['code_region'].astype(str)

In [759]:
base_regions.isna().sum()

exer                           0
outre_mer                      0
reg_is_ctu                     0
reg_code                       0
reg_name                       0
categ                          0
siren                          0
lbudg                          0
agregat                        0
montant_bp                     0
montant_ba                     0
montant_flux                   0
montant                        0
montant_en_millions            0
ptot                           0
euros_par_habitant             0
ordre_analyse1_section1    11951
ordre_analyse1_section2    11951
ordre_analyse1_section3    12431
ordre_analyse2_section1    11711
ordre_analyse2_section2    11711
ordre_analyse2_section3    12431
ordre_analyse3_section1    12191
ordre_analyse3_section2    12191
ordre_analyse3_section3    12191
ordre_analyse4_section1    12671
annee_join                     0
dtype: int64

In [760]:
valeurs_dict_reg = base_regions.groupby('siren')['reg_code'].first().to_dict()
communities_df['code_region'] = communities_df['code_region'].fillna(communities_df['siren'].map(valeurs_dict_reg))


In [761]:
# aucune valeur null pour code region
communities_df.isna().sum()

nom                               0
siren                             0
type                              0
cog                            1263
code_departement                115
code_departement_3digits        115
code_region                       0
population                        0
epci                           1424
trancheeffectifsunitelegale    2104
effectifssup50                    0
id                                0
dtype: int64

## Code département

### Formatage

In [762]:
communities_df['code_departement'] = communities_df['code_departement'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)


In [763]:
import re
pattern = r'^([02][1-9]|2[AB]|[1345678][0-9]|9[012345]|97[12346])$'

communities_df['valide'] = communities_df['code_departement'].astype(str).apply(lambda x:  bool(re.match(pattern, x)))

In [764]:
communities_df['valide'].value_counts()

True     36140
False      205
Name: valide, dtype: int64

In [765]:
df_erreurs = communities_df[~communities_df['valide']]
df_erreurs

,nom,siren,type,cog,code_departement,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,id,valide
id,,,,,,,,,,,,,
reg239710015,Guadeloupe,239710015,REG,1,NaN,NaN,nan,396153.0,NaN,42.0,True,reg239710015,False
reg237500079,Île-de-France,237500079,REG,11,NaN,NaN,nan,12291279.0,NaN,53.0,True,reg237500079,False
ctu200055507,Martinique,200055507,CTU,2,NaN,NaN,nan,377711.0,NaN,51.0,True,ctu200055507,False
reg234500023,Centre-Val de Loire,234500023,REG,24,NaN,NaN,nan,2634852.0,NaN,51.0,True,reg234500023,False
reg200053726,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,nan,2885864.0,NaN,51.0,True,reg200053726,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
cc248200107,CC du Quercy Rouergue et des Gorges de l'Aveyron,248200107,CC,NaN,"81,82","81,82",76,7827.0,NaN,NaN,False,cc248200107,False
ca248400251,CA du Grand Avignon (COGA),248400251,CA,NaN,"30,84","30,84","76,93",196849.0,NaN,NaN,False,ca248400251,False
cc248400335,CC Vaison Ventoux,248400335,CC,NaN,"26,84","26,84","84,93",17120.0,NaN,NaN,False,cc248400335,False


In [766]:
df_erreurs['type'].value_counts()

DEP    98
CC     67
CA     20
REG    14
CTU     3
MET     2
CU      1
Name: type, dtype: int64

In [767]:
df_erreurs[df_erreurs['type'] == "MET"]

,nom,siren,type,cog,code_departement,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,id,valide
id,,,,,,,,,,,,,
met200054781,Métropole du Grand Paris,200054781,MET,NaN,"75,91,92,93,94,95","75,91,92,93,94,95",11,7109664.0,NaN,NaN,False,met200054781,False
met200054807,Métropole d'Aix-Marseille-Provence,200054807,MET,NaN,"13,83,84","13,83,84",93,1900023.0,NaN,NaN,False,met200054807,False


In [768]:
communities_df = communities_df.drop(columns='valide')

##### Remarque : Les valeurs qui ne passe pas sont des codes de type metropole grand paris'75,91,92,93,94,95'\
##### et dans le cas Alsace on a deux ligne avec des siren différentes qui ont type DEP! avec comme code département 67A,\
##### est ce que c 'est correct

### Compléter code département
source nocodb :ofgl-base-departements-consolidee.csv

In [769]:
bases_departements = pd.read_csv(path +'ofgl-base-departements-consolidee.csv', delimiter=';')
bases_departements

,exer,outre_mer,reg_code,reg_name,dep_tranche_population,dep_status,dep_code,dep_name,categ,siren,...,ordre_analyse1_section3,ordre_analyse2_section1,ordre_analyse2_section2,ordre_analyse2_section3,ordre_analyse3_section1,ordre_analyse3_section2,ordre_analyse3_section3,ordre_analyse4_section1,annee_join,ptot_n
0,2018,Non,76,Occitanie,1,rural,81,Tarn,DEPT,228100012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,399108
1,2018,Non,75,Nouvelle-Aquitaine,1,rural,79,Deux-Sèvres,DEPT,227900016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,385029
2,2018,Non,75,Nouvelle-Aquitaine,1,rural,24,Dordogne,DEPT,222400012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,423868
3,2018,Non,84,Auvergne-Rhône-Alpes,3,urbain,74,Haute-Savoie,DEPT,227400017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,838044
4,2018,Non,11,Île-de-France,4,urbain,91,Essonne,DEPT,229102280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,1311028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75514,2020,Non,11,Île-de-France,4,urbain,75,Paris,PARIS,217500016,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2020,2192485
75515,2020,Non,11,Île-de-France,4,urbain,75,Paris,PARIS,217500016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2192485
75516,2020,Non,11,Île-de-France,4,urbain,75,Paris,PARIS,217500016,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2020,2192485
75517,2020,Non,11,Île-de-France,4,urbain,75,Paris,PARIS,217500016,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2192485


In [770]:
bases_departements.isna().sum()

exer                           0
outre_mer                      0
reg_code                       0
reg_name                       0
dep_tranche_population         0
dep_status                     0
dep_code                       0
dep_name                       0
categ                          0
siren                          0
lbudg                          0
agregat                        0
montant_bp                     0
montant_ba                     0
montant_flux                   0
montant                        0
montant_en_millions            0
ptot                           0
euros_par_habitant             0
ordre_analyse1_section1    69569
ordre_analyse1_section2    69569
ordre_analyse1_section3    71949
ordre_analyse2_section1    68379
ordre_analyse2_section2    68379
ordre_analyse2_section3    71949
ordre_analyse3_section1    70759
ordre_analyse3_section2    70759
ordre_analyse3_section3    70759
ordre_analyse4_section1    73139
annee_join                     0
ptot_n    

In [771]:
bases_departements['siren'] = bases_departements['siren'].apply(clean_numeros)

In [784]:
valeurs_dict_dep = bases_departements.groupby('siren')['dep_code'].first().to_dict()
communities_df['code_departement'] = communities_df['code_departement'].fillna(communities_df['siren'].map(valeurs_dict_dep))

In [773]:
communities_df.isna().sum()

nom                               0
siren                             0
type                              0
cog                            1263
code_departement                 17
code_departement_3digits        115
code_region                       0
population                        0
epci                           1424
trancheeffectifsunitelegale    2104
effectifssup50                    0
id                                0
dtype: int64

In [774]:
communities_df[communities_df["code_departement"].isnull()]["type"].value_counts()

REG    14
CTU     3
Name: type, dtype: int64

### Remarque
Le code département est null quand il s'agit d'une region et CTU ce qui correct

## Code cog


In [775]:
communities_df.isna().sum()

nom                               0
siren                             0
type                              0
cog                            1263
code_departement                 17
code_departement_3digits        115
code_region                       0
population                        0
epci                           1424
trancheeffectifsunitelegale    2104
effectifssup50                    0
id                                0
dtype: int64

#### Formatage

In [776]:
communities_df['cog'] = communities_df['cog'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)


### Compléter le code cog

#### Remarque: j 'ai regardé d'autre sources:
https://static.data.gouv.fr/resources/identifiants-des-collectivites-territoriales-et-leurs-etablissements/20221027-114559/identifiants-regions-2022.csv \
https://static.data.gouv.fr/resources/identifiants-des-collectivites-territoriales-et-leurs-etablissements/20221027-114720/identifiants-departements-2022.csv\

https://static.data.gouv.fr/resources/identifiants-des-collectivites-territoriales-et-leurs-etablissements/20221027-114832/identifiants-communes-2022.csv\

le code cog n'est pas renseigné dans le cas des types cc,ca, met, cu et ept comme le fichier initial 'all_communities"

In [777]:
communities_df[communities_df['cog'].notna()].duplicated().sum()

0

##### A continuer plus tard sur les champs (epci,trancheeffectifsunitelegale,effectifssup50) si c 'est necessaire (ou les supprimer).

## Ajout latitude et longitude

    


In [778]:
bases_geoloc_reg_dep = pd.read_csv(path + 'dep_reg_centers.csv', delimiter=';')

### Latitude et longitude des regions et départements
* **Sources**: 
    * Nocodb: 'Coordonnées géographiques - Départements et Régions' contient  que regions et départements

In [779]:
bases_geoloc_reg_dep

,type,cog,nom,longitude,latitude
0,REG,44,Alsace Champagne-Ardenne Lorraine,"5,619444","48,689167"
1,REG,75,Aquitaine Limousin Poitou-Charentes,"0,197778","45,192222"
2,REG,84,Auvergne Rhône-Alpes,"4,538056","45,515833"
3,REG,27,Bourgogne Franche-Comté,"4,809167","47,235278"
4,REG,53,Bretagne,"-1,161389","48,179722"
...,...,...,...,...,...
114,DEP,67A,Alsace,"7,551389","48,670833"
115,DEP,67A,Alsace,"7,551389","48,670833"
116,DEP,971,Guadeloupe,"-61,568686","16,252882"
117,DEP,974,La Réunion,"-21,114533","55,532062"


In [780]:
communities_df = communities_df.merge(bases_geoloc_reg_dep[['cog', 'type', 'longitude', 'latitude']], on=['cog', 'type'], how='left')

In [781]:
communities_df

,nom,siren,type,cog,code_departement,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,id,longitude,latitude
0,Guadeloupe,239710015,REG,1,NaN,NaN,nan,396153.0,NaN,42.0,True,reg239710015,"-61,568686","16,252882"
1,Île-de-France,237500079,REG,11,NaN,NaN,nan,12291279.0,NaN,53.0,True,reg237500079,"2,504722","48,709167"
2,Martinique,200055507,CTU,2,NaN,NaN,nan,377711.0,NaN,51.0,True,ctu200055507,"-61,015827","14,636792"
3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,nan,2634852.0,NaN,51.0,True,reg234500023,"1,685278","47,480556"
4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,nan,2885864.0,NaN,51.0,True,reg200053726,"4,809167","47,235278"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36342,CA CIVIS (Communauté Intercommunale des VIlles...,249740077,CA,NaN,974,974,04,182463.0,NaN,NaN,False,ca249740077,NaN,NaN
36343,CA du Sud,249740085,CA,NaN,974,974,04,130153.0,NaN,NaN,False,ca249740085,NaN,NaN
36344,CA Intercommunale de la Réunion Est (CIREST),249740093,CA,NaN,974,974,04,128237.0,NaN,NaN,False,ca249740093,NaN,NaN
36345,CA Territoire de la Côte Ouest (TCO),249740101,CA,NaN,974,974,04,214963.0,NaN,NaN,False,ca249740101,NaN,NaN
